In [3]:
import os
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def raw_data_formatter(file_list,file_dir):
    df = pd.DataFrame()

    for arquivo in file_list:
        file = os.path.join(file_dir, arquivo)
        date=datetime.strptime(arquivo.split(sep='.')[0],'%m-%d-%Y')
        df_arquivo = pd.read_csv(file)
        df_arquivo['Date'] = date
        df = pd.concat([df,df_arquivo])
        
    # Merging the data from columns with same content but different headers:
    Country = df.Country_Region
    Province = df.Province_State
    Last_Update = df.Last_Update
    Latitude = df.Lat
    Longitude = df.Long_

    df_aux = pd.DataFrame({'Country/Region': Country,'Province/State': Province,'Last Update': Last_Update,'Latitude': Latitude,
                           'Longitude': Longitude})
    df = df.combine_first(df_aux)

    # Dropping columns that won't be used:
    df.drop(axis=1,labels=['Country_Region','Province_State','Last_Update','FIPS',
                           'Combined_Key','Long_','Lat','Admin2','Case-Fatality_Ratio',
                           'Incidence_Rate'],inplace=True)

    # Formatting datetime columns:
    df['Last Update'] = pd.to_datetime(df['Last Update'])

    # Replacing NaN values on numeric data with 0:
    new_values = {'Deaths': 0, 'Active': 0, 'Recovered': 0, 'Confirmed': 0, 'Latitude': 0, 'Longitude': 0}
    df.fillna(value=new_values,inplace=True)

    # Replacing NaN values on non numeric data with '-':
    df.fillna(value='-',inplace=True)

    # Adding date columns:
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.to_datetime(df['Date']).dt.strftime('%b')
    df['Week'] = pd.DatetimeIndex(df['Date']).week
    df['Day'] = pd.DatetimeIndex(df['Date']).day

    # Establishing number of active cases as the difference between Confirmed cases and Death cases:
    df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

    # Calculating Mortality rate as the ratio between Deaths and Confirmed cases for each day:
    df['Mortality rate in %'] = (df['Deaths']/df['Confirmed']*100).fillna(value=0)
    
    return df

In [3]:
cwd = os.getcwd()
who_data_dir = r"csse_covid_19_data\csse_covid_19_daily_reports"
raw_data_dir = cwd

# Read the data update information file to check
# for the last update:

update_info = pd.read_csv(os.path.join(raw_data_dir,'raw_data_info.csv'))
last_update = pd.to_datetime(update_info.last_update.values[0])

# Read the data files in the WHO Github directory:

who_file_list = os.listdir(os.path.join(cwd,who_data_dir))

for file in who_file_list:
    if not file.endswith('.csv'):
        who_file_list.remove(file)

# Now compares the latest WHO file to the raw data update information
# and generates a boolean value for wether they match or not:

latest_who_file_date = pd.to_datetime(who_file_list[-1].split(sep='.')[0])
files_to_update = (latest_who_file_date - last_update).days

if files_to_update != 0:
    list_of_new_files = []
    for i in list(range(1,files_to_update + 1)):
        new_date = datetime.strftime((last_update
                                      + timedelta(days=i)).date(), format='%m-%d-%Y')
        list_of_new_files.append(new_date + '.csv')
    
    # Generating a dataframe with new information:
    df = raw_data_formatter(list_of_new_files,who_data_dir)
    
    # Appending the new data to existing raw data file and updating
    # the raw data info file:
    df.to_csv('raw_data.csv', mode='a', index=False, header=None)
    update_info.last_update = new_date
    update_info.to_csv('raw_data_info.csv', index=False)

In [10]:
df=pd.DataFrame({'file':['raw_data','raw_data_info','who_data'],
                 'path':[r"C:\Users\user\Documents\GitHub\COVID-19\notebooks\raw_data.csv",
                         r"C:\Users\user\Documents\GitHub\COVID-19\notebooks\raw_data_info.csv",
                         r"C:\Users\user\Documents\GitHub\COVID-19\csse_covid_19_data\csse_covid_19_daily_reports"]})

In [2]:
df.to_csv('file_path.csv',index=False)

NameError: name 'df' is not defined

In [11]:
df = pd.read_csv('config.csv',index_col='var').fillna('-')

In [12]:
df

,path,last_update
var,,
raw_data,C:\Users\user\Documents\GitHub\COVID-19\notebo...,07-13-2020
who_data_dir,C:\Users\user\Documents\GitHub\COVID-19\csse_c...,-


In [33]:
os.path.join(df.loc['who_data_dir'].path,'07-09-2020.csv')

'C:\\Users\\user\\Documents\\GitHub\\COVID-19\\csse_covid_19_data\\csse_covid_19_daily_reports\\07-09-2020.csv'